### Введение в группировку

Основа любой агрегации в SQL — это оператор `GROUP BY`. Он позволяет сгруппировать строки с одинаковыми значениями в указанных столбцах и применить к каждой группе агрегатные функции (`SUM`, `COUNT`, `AVG` и т.д.).

**Простой пример:** Посчитаем, сколько заказов сделал каждый клиент.

```sql
SELECT
    customer_id,
    COUNT(order_id) AS total_orders
FROM
    orders
GROUP BY
    customer_id
ORDER BY
    total_orders DESC;
```

Этот запрос "схлопывает" все строки для каждого `customer_id` в одну и считает количество заказов. Но что, если нам нужны более сложные срезы данных в одном отчёте? Например, итоги по странам, по городам и общий итог? Делать три отдельных запроса и объединять их через `UNION ALL` неудобно и неэффективно. Здесь на помощь приходят продвинутые инструменты группировки.

-----

### GROUPING SETS, ROLLUP, CUBE

Это мощные расширения `GROUP BY`, которые позволяют создавать сложные многоуровневые отчёты одним запросом.

#### **`GROUPING SETS`**: Гибкая настройка группировок

`GROUPING SETS` позволяет явно указать несколько наборов столбцов, по которым нужно сгруппировать данные. Это как выполнить несколько `GROUP BY` запросов одновременно.

**Пример:**
Представим, что отделу аналитики нужен отчёт по продажам (таблица `order_details` и `products`) со следующими срезами:

1.  Общая сумма продаж по каждой **категории товаров**.
2.  Общая сумма продаж для каждого **поставщика** (`supplier`).
3.  Общий итог по всем продажам.

<!-- end list -->

```sql
SELECT
    c.category_name,
    s.company_name AS supplier_name,
    SUM(od.unit_price * od.quantity) AS total_sales
FROM
    order_details od
JOIN
    products p ON od.product_id = p.product_id
JOIN
    categories c ON p.category_id = c.category_id
JOIN
    suppliers s ON p.supplier_id = s.supplier_id
GROUP BY
    GROUPING SETS (
        (c.category_name),      -- Группировка №1: по категории
        (s.company_name),       -- Группировка №2: по поставщику
        ()                      -- Группировка №3: общий итог (пустой набор)
    )
ORDER BY
    c.category_name, s.company_name;
```

**Как это работает:**

  - `GROUPING SETS` принимает список "наборов группировки".
  - В результате мы увидим строки, где посчитана сумма `total_sales` отдельно для каждой категории (`supplier_name` будет `NULL`), отдельно для каждого поставщика (`category_name` будет `NULL`) и одна строка с общим итогом (и `category_name`, и `supplier_name` будут `NULL`).

`NULL` в столбце группировки здесь означает, что по этому полю агрегация не производилась (это подитог).

-----

#### **`ROLLUP`**: Иерархические итоги 📈

`ROLLUP` — это сокращённая запись для `GROUPING SETS`, предназначенная для иерархических данных. `ROLLUP (a, b, c)` эквивалентен `GROUPING SETS ((a, b, c), (a, b), (a), ())`. Он идеально подходит для данных, где есть вложенность, например: Страна -\> Город -\> Клиент.

**Пример:**
Построим иерархический отчёт о количестве заказов по географии:

1.  Количество заказов для каждого клиента в каждом городе.
2.  Подитог по каждому городу.
3.  Подитог по каждой стране.
4.  Общий итог.

<!-- end list -->

```sql
SELECT
    c.country,
    c.city,
    c.company_name,
    COUNT(o.order_id) AS total_orders
FROM
    customers c
JOIN
    orders o ON c.customer_id = o.customer_id
GROUP BY
    ROLLUP (c.country, c.city, c.company_name)
ORDER BY
    c.country, c.city, c.company_name;
```

**Результат:**
Вы получите детальные данные, а также строки с `NULL`, обозначающие подитоги. Например, строка, где `company_name` равен `NULL`, но `country` и `city` заполнены, будет содержать общее число заказов для этого города. Строка, где `NULL` везде, — это общий итог по всем заказам.

-----

#### **`CUBE`**: Многомерный анализ 🧊

`CUBE` — самый мощный инструмент. Он создаёт итоги для **всех возможных комбинаций** столбцов. `CUBE (a, b)` эквивалентен `GROUPING SETS ((a, b), (a), (b), ())`. Это идеально для построения "OLAP-кубов" данных.

**Пример:**
Проанализируем объём проданных товаров (`quantity`) в разрезе **категории товара** и **страны заказчика**. Нам нужны все возможные комбинации итогов.

```sql
SELECT
    c.category_name,
    cust.country,
    SUM(od.quantity) as total_quantity
FROM
    order_details od
JOIN
    products p ON od.product_id = p.product_id
JOIN
    categories c ON p.category_id = c.category_id
JOIN
    orders o ON od.order_id = o.order_id
JOIN
    customers cust ON o.customer_id = cust.customer_id
WHERE
    cust.country IN ('Germany', 'USA', 'UK') -- Ограничим для наглядности
GROUP BY
    CUBE (c.category_name, cust.country)
ORDER BY
    c.category_name, cust.country;
```

**Результат:**
Этот запрос вернёт:

  - Продажи по каждой категории в каждой стране.
  - Итоги по каждой категории (для всех стран).
  - Итоги по каждой стране (для всех категорий).
  - Общий итог.

-----